<a href="https://colab.research.google.com/github/aknip/Autogen-AutoGPT/blob/main/AutoGen_Screen_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogen Data Scientist Coder Group

Notes:
- Sometimes code execution fails?
- Source: https://medium.com/microsoftazure/introducing-autogen-df7fa6cb81ee

In [1]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [ ]:
!pip install pyautogen~=0.1.0 docker

## Setup




In [5]:
from autogen import AssistantAgent, UserProxyAgent
import autogen

config_list = [{'model': 'gpt-4', 'api_key': CREDS['OpenAI']['v1']['credential']}]

#print(config_list)

llm_config = {
    "seed": 43,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    system_message="You follow an approved plan. You write python/shell code to solve tasks and extract information from arxiv. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor. Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor. If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.",
    llm_config=llm_config,
)
data_scientist = autogen.AssistantAgent(
    name="data_scientist",
    system_message="Expert in assessing Machine Learning models.",
    llm_config=llm_config,
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin, until admin approval.
The plan may involve a coder who can write code and a data_scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by a coder, and which step is performed by a data_scientist.
''',
    llm_config=llm_config,
)
executor = autogen.UserProxyAgent(
   name="executor",
   system_message="Execute code produced by the coder in the groupchat folder.",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"}
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, data_scientist, planner, executor], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


# Start Chat

In [6]:
user_proxy.initiate_chat(
    manager,
    message="""
    Find information about computer vision models on arxiv and generate a comparison tables in markdown of the three most performing models.
    Make sure to highlight also concrete applications in the medical industry.
    """)

User_proxy (to chat_manager):


    Find information about computer vision models on arxiv and generate a comparison tables in markdown of the three most performing models.
    Make sure to highlight also concrete applications in the medical industry.
    

--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. **Data Scientist**: Start by conducting a literature review on arXiv, focusing on the most recent and relevant papers on computer vision models. The data scientist will use their knowledge and expertise to identify the three most performing models based on the information provided in the papers.

2. **Data Scientist**: After identifying the top three models, the data scientist will then delve deeper into each model, gathering information about their performance metrics, advantages, disadvantages, and any specific applications in the medical industry.

3. **Data Scientist**: The data scientist will then compile this 